In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [2]:
mnist = input_data.read_data_sets('MNIST/', one_hot=True)
train_images = mnist.train.images
train_labels = mnist.train.labels
test_images = mnist.test.images
test_labels = mnist.test.labels
print('MNIST ready')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
MNIST ready


In [3]:
n_input = 784
n_output = 10
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.1)),
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.1)),
    'wd1': tf.Variable(tf.random_normal([7*7*128, 1024], stddev=0.1)),
    'wd2': tf.Variable(tf.random_normal([1024, n_output], stddev=0.1))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bc2': tf.Variable(tf.random_normal([128], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([1024], stddev=0.1)),
    'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1)),
    
}

In [4]:
def conv_basic(_input, _w, _b, _keepratio):
    
    _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
    # CONV LAYER 1
    _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1, _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1], strides=[1, 2,2, 1], padding='SAME')
    _pool1_dr1 = tf.nn.dropout(_pool1, _keepratio)
    
    # CONV layer2
    _conv2 = tf.nn.conv2d(_pool1_dr1, _w['wc2'], strides=[1, 1, 1, 1], padding='SAME')
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2, _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    _pool2_dr2 = tf.nn.dropout(_pool2, _keepratio)
    
    # vectorize
    _densel = tf.reshape(_pool2_dr2, [-1, _w['wd1'].get_shape().as_list()[0]])
    # Fully connected layer1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_densel, _w['wd1']), _b['bd1']))
    _fc1_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # Fully connected layer2
    _out = tf.add(tf.matmul(_fc1_dr1, _w['wd2']), _b['bd2'])
    out = {
        'input_r': _input_r,
        'conv1': _conv1,
        'conv2': _conv2,
        'pool1': _pool1,
        'pool2': _pool2,
        'pool1_dr1': _pool1_dr1,
        'pool2_dr2': _pool2_dr2,
        'densel': _densel,
        'fc1': _fc1,
        'fc1_dr1': _fc1_dr1,
        'out': _out
    }
    return out
print('CNN ready')

CNN ready


In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

# functions
pred = conv_basic(x, weights, biases, keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.arg_max(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(_corr, tf.float32))
init = tf.global_variables_initializer()

# saver
save_step = 1
saver = tf.train.Saver(max_to_keep=3)
print('Graph ready')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `argmax` instead
Graph ready


In [12]:
do_train = 1
sess = tf.Session()
sess.run(init)

In [17]:
train_epochs = 15
batch_size = 16
display_step = 1
if do_train == 1:
    for epoch in range(train_epochs):
        start_time = time.time()
        avg_cost = 0
        num_batch = int(mnist.train.num_examples/batch_size)
        for i in range(num_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feeds = {x: batch_xs, y: batch_ys, keepratio: 0.7}
            sess.run(optm, feed_dict=feeds)
            avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
        if epoch % display_step == 0:
            train_accr = sess.run(accr, feed_dict=feeds)
            feeds_test = {x: mnist.test.images, y: mnist.test.labels, keepratio: 1.}
            test_accr = sess.run(accr, feed_dict=feeds_test)
            print('Eopch: %5d  train_accr: %.8f    test_acc:%.8f   loss:%.5f' %(epoch, train_accr, test_accr, avg_cost))
            end_time = time.time()
            print('Time is:', end_time-start_time)

Eopch:     0  train_accr: 0.93750000    test_acc:0.98769999   loss:0.07436
Time is: 317.03809571266174
Eopch:     1  train_accr: 1.00000000    test_acc:0.98780000   loss:0.05172
Time is: 325.8013997077942
Eopch:     2  train_accr: 1.00000000    test_acc:0.99159998   loss:0.04396
Time is: 327.81976890563965
Eopch:     3  train_accr: 1.00000000    test_acc:0.99220002   loss:0.03706
Time is: 327.553058385849
Eopch:     4  train_accr: 1.00000000    test_acc:0.99159998   loss:0.03163
Time is: 328.1135501861572
Eopch:     5  train_accr: 1.00000000    test_acc:0.99250001   loss:0.02681
Time is: 322.01633620262146
Eopch:     6  train_accr: 1.00000000    test_acc:0.99229997   loss:0.02808
Time is: 322.6861152648926
Eopch:     7  train_accr: 1.00000000    test_acc:0.99290001   loss:0.02662
Time is: 326.4110221862793
Eopch:     8  train_accr: 1.00000000    test_acc:0.99299997   loss:0.02809
Time is: 328.5777826309204
Eopch:     9  train_accr: 1.00000000    test_acc:0.99080002   loss:0.02368
Time 